In [1]:
import os
os.environ["DGLBACKEND"] = "pytorch"

In [2]:
import torch 
import pandas as pd
import dgl
from dgl.data import DGLDataset
import dgl.function as fn
import numpy as np
import xarray as xr

In [3]:
ds = dgl.data.CSVDataset('toy_dataset/')
g = ds[0]

Done loading data from cached files.


In [4]:
g.ndata['k'] = torch.rand(g.num_nodes())*1000
g.ndata['alpha'] = torch.rand(g.num_nodes())+1
TechTable = np.array([[0.3,0],[0.35,0.15],[0.45, 0.65]])

g.ndata['income'] = torch.max((g.ndata['alpha'][:,None]*g.ndata['k'][:,None]**TechTable[:,0] - TechTable[:,1]), axis=1)[0]
g.ndata['income']

tensor([27.4085, 22.3742, 24.9988, 17.1449, 24.3291, 27.9562, 24.4836, 14.8005,
        40.7935, 16.4986], dtype=torch.float64)

In [9]:
g.ndata['income'].dtype

torch.float64

In [82]:
agent_data_instance = xr.Dataset()

In [62]:
ndata = list(g.node_attr_schemes().keys())

In [63]:
for prop in ndata:
    agent_data_instance = agent_data_instance.assign(prop=(['n_agents','ntime'], g.ndata[prop][:,None].numpy()))
    agent_data_instance = agent_data_instance.rename(name_dict={'prop':prop})

In [65]:
agent_data_instance

<xarray.Dataset>
Dimensions:  (n_agents: 10, ntime: 1)
Dimensions without coordinates: n_agents, ntime
Data variables:
     na2     (n_agents, ntime) int64 1 2 3 4 5 6 7 8 9 10
     na1     (n_agents, ntime) int64 1 2 3 4 5 6 7 8 9 10
    k        (n_agents, ntime) float32 460.9 877.9 938.2 ... 750.3 627.1 889.3
    alpha    (n_agents, ntime) float32 1.319 1.644 1.547 ... 1.753 1.542 1.077
    income   (n_agents, ntime) float64 20.2 34.06 33.0 ... 33.83 27.34 22.21

In [81]:
agent_data_instance.to_zarr('./test.zarr', append_dim='ntime')

ValueError: append_dim='ntime' does not match any existing dataset dimensions {}

In [78]:
import zarr 
test = zarr.load('./test')

In [80]:
test['k']

array([[460.92148 , 460.92148 ],
       [877.9074  , 877.9074  ],
       [938.2327  , 938.2327  ],
       [121.624115, 121.624115],
       [ 97.7748  ,  97.7748  ],
       [564.2234  , 564.2234  ],
       [844.94196 , 844.94196 ],
       [750.32605 , 750.32605 ],
       [627.0685  , 627.0685  ],
       [889.3475  , 889.3475  ]], dtype=float32)

In [88]:
from os.path import exists

exists('test2.zarr')

False

In [102]:
from pathlib import Path
epath = './edge_data'
timestep = 1
Path(epath)/(str(timestep)+'.zarr')

PosixPath('edge_data/1.zarr')